# Analysis for Trapnell's 271 primary human myoblasts

Aftering obtaining a transcript compatibility count (TCC) distribution matrix, we use the matrix to:

1. Cluster cells via affinity propagation
2. Generate minimum weight spanning trees with pie-charts as nodes
3. Generate the bar plots for visualizing expressions of certain genes within clusters

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pickle

## Initial Cleanup of Data

We cleanup the distance matrices to restrict ourselves to the 271 cells used by Trapnell et al.

In [14]:
cells_selected=np.loadtxt(
    "/data/SS_RNA_seq/Code/metadata/Trapnell_metadata/HSMM/HSMM_sample_names.txt",
    dtype=str)
cell_ids=np.loadtxt(
    "/data/SS_RNA_seq/Code/metadata/Trapnell_metadata/cell_ids_gk.txt",
    dtype=str)
cells_ids_SRR = np.loadtxt("/data/SS_RNA_seq/Code/metadata/Trapnell_metadata/SRR.txt"
                           ,dtype=str)
cells_not_selected_SRR = [cells_ids_SRR[i]+'.fastq.gz' 
                      for i in range(len(cells_ids_SRR)) 
                      if cell_ids[i] not in cells_selected]
good_cells=[i for i, j in enumerate(cell_ids) if j in cells_selected]



271


## Clustering

In [1]:
# Clustering is done using scikit-learn's implementation of affinity propagation

from sklearn import cluster
# D is a symmetric N-by-N distance matrix where N is the number of cells
def AffinityProp(D):
    aff= cluster.AffinityPropagation(affinity='precomputed')
    labels=aff.fit_predict(D)
    return labels

## MST

In [2]:
# 

## Bar plot